<a href="https://colab.research.google.com/github/kk412027247/nlp/blob/main/SMS_Spam_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
import os
import io
import pandas as pd
import re
tf.__version__

# path_to_zip = tf.keras.utils.get_file("smsspamcollection.zip",
#                                       origin = "https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip",extract=True)


# !unzip $path_to_zip -d data

lines = io.open('data/SMSSpamCollection').read().strip().split('\n')
lines[0]

spam_dataset = []
for line in lines:
  label, text = line.split('\t')
  if label.lower().strip() == 'spam':
    spam_dataset.append((1, text.strip()))
  else:
    spam_dataset.append(((0, text.strip())))
print(spam_dataset[0])

df = pd.DataFrame(spam_dataset, columns=['Spam', 'Message'])

def message_lenth(x):
  return len(x)

def num_capitals(x):
  _, count = re.subn(r'[A-Z]', '', x)
  return count

def num_puntuation(x):
  _,count = re.subn(r'\W', '', x)
  return count

df['Capitals'] = df['Message'].apply(num_capitals)
df['Punctuation'] = df['Message'].apply(num_puntuation)
df['Length'] = df['Message'].apply(message_lenth)
df.describe()

train=df.sample(frac=0.8, random_state=42)
test=df.drop(train.index)
x_train=train[['Length', 'Capitals', 'Punctuation']]
y_train = train[['Spam']]
x_test = test[['Length', 'Capitals', 'Punctuation']]
y_test = test[['Spam']]


def make_model(input_dims=3, num_units=12):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Dense(num_units,
                                  input_dim=input_dims,
                                  activation='relu'))
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

model = make_model()
model.fit(x_train, y_train, epochs=10, batch_size=10)

model.evaluate(x_test, y_test)

y_train_pred = model.predict(x_train)
x = tf.math.confusion_matrix(tf.constant(y_train.Spam), y_train_pred)
print(x)

sentence = 'Go until jurong point, crazy.. Available only in bugis n great world'
sentence.split()

# !pip install stanfordnlp
import stanfordnlp as snlp
# en = snlp.download('en')

# en = snlp.Pipeline(lang='en', processors='tokenize')

# tokenized = en(sentence)
# len(tokenized.sentences)

# for snt in tokenized.sentences:
#   for word in snt.tokens:
#     print(word.text)
#   print("<end of Sentence>")



# jp = snlp.download('ja')
# jp = snlp.Pipeline(lang='ja', processors='tokenize')
# jp_line = jp("選挙管理委員会")

# for snt in jp_line.sentences:
#   for word in snt.tokens:
#     print(word.text)



pipeline = snlp.Pipeline(lang='en', processors='tokenize')

def word_counts(x):
  doc = pipeline(x)
  count = sum( [ len(sentence.tokens) for sentence in doc.sentences] )
  return count



train['Words'] = train['Message'].apply(word_counts)
test['Words'] = test['Message'].apply(word_counts)

x_train=train[['Length', 'Capitals', 'Punctuation', 'Words']]
y_train = train[['Spam']]
x_test = test[['Length', 'Capitals', 'Punctuation', 'Words']]
y_test = test[['Spam']]

model = make_model(input_dims=4)

model.fit(x_train, y_train, epochs=10, batch_size=10)

print(train.loc[train.Spam == 1].describe())


print(train.loc[train.Spam == 0].describe())


